 ## Installation and Setup

In [1]:
# Install required packages
!pip install flask flask-cors requests termcolor pyngrok bitsandbytes -q

print("All packages installed successfully!")
print("Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.4 MB/s eta 0:00:00
All packages installed successfully!
Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/genai_hw_MCP_A2A

/content/drive/MyDrive/genai_hw_MCP_A2A


In [4]:
import json
from flask import Flask, request, Response, jsonify
from flask_cors import CORS
import sqlite3
import threading
import time
import requests
from dataclasses import dataclass, field
from typing import Dict, Any, List, Optional

from database_setup import DatabaseSetup

## Database Setup

In [5]:
def init_db(db_path: str = "support.db") -> sqlite3.Connection:
    """
    Initialize the SQLite database using DatabaseSetup
    and return a shared connection.
    """
    # Run setup (create tables, sample data, etc.)
    setup = DatabaseSetup(db_path=db_path)
    setup.connect()
    setup.create_tables()
    setup.create_triggers()
    setup.insert_sample_data()
    setup.close()

    conn = sqlite3.connect(db_path, check_same_thread=False)
    conn.row_factory = sqlite3.Row
    return conn

db_conn = init_db()

Connected to database: support.db
Tables created successfully!
Triggers created successfully!
Sample data inserted successfully!
  - 15 customers added
  - 25 tickets added
Database connection closed.


## Customer Management Functions

In [6]:
def get_customer(args: Dict[str, Any]) -> Dict[str, Any]:
    if isinstance(args, int):
        args = {"customer_id": args}

    customer_id = args.get("customer_id")
    if customer_id is None:
        return {"success": False, "error": "customer_id is required"}

    cur = db_conn.cursor()
    cur.execute("SELECT * FROM customers WHERE id = ?", (customer_id,))
    row = cur.fetchone()

    if not row:
        return {"success": False, "error": f"No customer found with id {customer_id}"}

    return {
        "success": True,
        "data": dict(row),
    }


def list_customers(args: Dict[str, Any]) -> Dict[str, Any]:
    status = args.get("status")   # e.g. 'active' or 'disabled'
    limit = args.get("limit", 10)

    query = "SELECT * FROM customers"
    params: List[Any] = []

    if status:
        query += " WHERE status = ?"
        params.append(status)

    query += " ORDER BY id LIMIT ?"
    params.append(limit)

    cur = db_conn.cursor()
    cur.execute(query, tuple(params))
    rows = cur.fetchall()

    return {
        "success": True,
        "count": len(rows),
        "customers": [dict(r) for r in rows],
    }


def update_customer(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Args should look like:
    {
      "customer_id": 5,
      "data": {
        "email": "new@email.com",
        "phone": "+1-555-0123",
        ...
      }
    }
    """
    customer_id = args.get("customer_id")
    data = args.get("data", {})

    if not customer_id:
        return {"success": False, "error": "customer_id is required"}
    if not data:
        return {"success": False, "error": "data dict is required"}

    # Only allow certain fields
    allowed_fields = {"name", "email", "phone", "status"}
    fields = [k for k in data.keys() if k in allowed_fields]

    if not fields:
        return {"success": False, "error": "No valid fields to update"}

    set_clause = ", ".join(f"{f} = ?" for f in fields)
    params = [data[f] for f in fields]
    params.append(customer_id)

    cur = db_conn.cursor()
    cur.execute(f"UPDATE customers SET {set_clause} WHERE id = ?", tuple(params))
    db_conn.commit()

    if cur.rowcount == 0:
        return {"success": False, "error": f"No customer found with id {customer_id}"}

    # Return updated customer
    return get_customer({"customer_id": customer_id})


def create_ticket(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Args:
    {
      "customer_id": 5,
      "issue": "Double charged",
      "priority": "high"
    }
    """
    customer_id = args.get("customer_id")
    issue = args.get("issue")
    priority = args.get("priority", "medium")

    if not customer_id:
        return {"success": False, "error": "customer_id is required"}
    if not issue:
        return {"success": False, "error": "issue is required"}

    # Validate priority
    if priority not in ("low", "medium", "high"):
        return {"success": False, "error": "priority must be one of: low, medium, high"}

    cur = db_conn.cursor()

    # Ensure customer exists
    cur.execute("SELECT 1 FROM customers WHERE id = ?", (customer_id,))
    if not cur.fetchone():
        return {"success": False, "error": f"No customer found with id {customer_id}"}

    cur.execute(
        """
        INSERT INTO tickets (customer_id, issue, status, priority)
        VALUES (?, ?, 'open', ?)
        """,
        (customer_id, issue, priority),
    )
    db_conn.commit()

    ticket_id = cur.lastrowid

    cur.execute("SELECT * FROM tickets WHERE id = ?", (ticket_id,))
    ticket_row = cur.fetchone()

    return {
        "success": True,
        "ticket": dict(ticket_row),
    }


def get_customer_history(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Args:
    {
      "customer_id": 5
    }
    """
    customer_id = args.get("customer_id")
    if not customer_id:
        return {"success": False, "error": "customer_id is required"}

    cur = db_conn.cursor()
    cur.execute(
        """
        SELECT * FROM tickets
        WHERE customer_id = ?
        ORDER BY created_at DESC
        """,
        (customer_id,),
    )
    rows = cur.fetchall()

    return {
        "success": True,
        "count": len(rows),
        "tickets": [dict(r) for r in rows],
    }

In [7]:
# Test the functions
print("Customer management functions defined successfully!")
print("\n Available functions:")
print("   - get_customer(customer_id)")
print("   - list_customers(status, limit)")
print("   - update_customer(customer_id, data)")
print("   - create_ticket(customer_id, issue, priority)")
print("   - get_customer_history(customer_id)")

# Quick test
print("\n Quick test - Fetching customer ID 1:")
result = get_customer(1)
if result['success']:
    customer = result['data']
    print(f"   Name: {customer['name']}")
    print(f"   Email: {customer['email']}")
    print(f"   Status: {customer['status']}")

# Test list_customers
print("\n TEST – list_customers (limit 3):")
result = list_customers({"limit": 3})
if result["success"]:
    for c in result["customers"]:
        print(f" - {c['id']} | {c['name']} | {c['status']}")
else:
    print(result["error"])

# Test update_customer
print("\n TEST – update_customer (change email & phone for ID 1):")
result = update_customer({
    "customer_id": 1,
    "data": {
        "email": "john.new@example.com",
        "phone": "+1-555-7777"
    }
})
print(result)

# Test create_ticket
print("\n TEST – create_ticket for customer 1:")
result = create_ticket({
    "customer_id": 1,
    "issue": "Unable to login",
    "priority": "high"
})
print(result)

# Test get_customer_history
print("\n TEST – get_customer_history for customer 1:")
result = get_customer_history({"customer_id": 1})
if result["success"]:
    for t in result["tickets"]:
        print(f" - Ticket #{t['id']} | {t['issue']} | {t['status']} | {t['priority']}")
else:
    print(result["error"])

Customer management functions defined successfully!

 Available functions:
   - get_customer(customer_id)
   - list_customers(status, limit)
   - update_customer(customer_id, data)
   - create_ticket(customer_id, issue, priority)
   - get_customer_history(customer_id)

 Quick test - Fetching customer ID 1:
   Name: John Doe
   Email: john.new@example.com
   Status: active

 TEST – list_customers (limit 3):
 - 1 | John Doe | active
 - 2 | Jane Smith | active
 - 3 | Bob Johnson | disabled

 TEST – update_customer (change email & phone for ID 1):
{'success': True, 'data': {'id': 1, 'name': 'John Doe', 'email': 'john.new@example.com', 'phone': '+1-555-7777', 'status': 'active', 'created_at': '2025-11-30 20:26:34', 'updated_at': '2025-12-01 01:57:04'}}

 TEST – create_ticket for customer 1:
{'success': True, 'ticket': {'id': 86, 'customer_id': 1, 'issue': 'Unable to login', 'status': 'open', 'priority': 'high', 'created_at': '2025-12-01 01:57:04'}}

 TEST – get_customer_history for customer

## MCP HTTP Streaming Server Implementation

In [8]:
# ================================
# MCP Server
# ================================

# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Server state
server_thread = None
server_running = False

# =====================
# MCP Tool Definitions
# =====================

MCP_TOOLS = [
    {
        "name": "get_customer",
        "description": "Retrieve a specific customer by their ID. Returns customer details including name, email, phone, and status.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to retrieve"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "list_customers",
        "description": "List customers in the database. Can optionally filter by status and limit the number of results.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["active", "disabled"],
                    "description": "Optional filter by customer status"
                },
                "limit": {
                    "type": "integer",
                    "description": "Maximum number of customers to return (default 10)"
                }
            }
        }
    },
    {
        "name": "update_customer",
        "description": "Update an existing customer's information. Provide the customer ID and any fields you want to change.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to update"
                },
                "name": {
                    "type": "string",
                    "description": "New name (optional)"
                },
                "email": {
                    "type": "string",
                    "description": "New email (optional)"
                },
                "phone": {
                    "type": "string",
                    "description": "New phone (optional)"
                },
                "status": {
                    "type": "string",
                    "enum": ["active", "disabled"],
                    "description": "New status (optional)"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "create_ticket",
        "description": "Create a support ticket for a specific customer.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "ID of the customer who opened the ticket"
                },
                "issue": {
                    "type": "string",
                    "description": "Description of the customer's issue"
                },
                "priority": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Ticket priority (default: medium)"
                }
            },
            "required": ["customer_id", "issue"]
        }
    },
    {
        "name": "get_customer_history",
            "description": "Get all tickets associated with a given customer, ordered by most recent first.",
            "inputSchema": {
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "integer",
                        "description": "ID of the customer whose ticket history to fetch"
                    }
                },
                "required": ["customer_id"]
            }
    }
]

# ==========================
# SSE helper
# ==========================

def create_sse_message(data: Dict[str, Any]) -> str:
    """
    Format a message for Server-Sent Events (SSE).
    SSE format: 'data: {json}\\n\\n'
    """
    return f"data: {json.dumps(data)}\n\n"

# ==================================
# MCP handlers (initialize / list)
# ==================================

def handle_initialize(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle MCP initialize request.
    This is the first message in the MCP protocol handshake.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "protocolVersion": "2024-11-05",
            "capabilities": {
                "tools": {},
            },
            "serverInfo": {
                "name": "customer-support-mcp-server",
                "version": "1.0.0"
            }
        }
    }

def handle_tools_list(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/list request.
    Returns the list of available tools.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "tools": MCP_TOOLS
        }
    }

# ==================================
# Tool adapters
# ==================================

def mcp_get_customer(customer_id: int) -> Dict[str, Any]:
    """Adapter for get_customer(args)."""
    return get_customer({"customer_id": customer_id})

def mcp_list_customers(status: str = None, limit: int = 10) -> Dict[str, Any]:
    """Adapter for list_customers(args)."""
    args: Dict[str, Any] = {"limit": limit}
    if status:
        args["status"] = status
    return list_customers(args)

def mcp_update_customer(
    customer_id: int,
    name: str = None,
    email: str = None,
    phone: str = None,
    status: str = None,
) -> Dict[str, Any]:
    """Adapter for update_customer(args) """
    data: Dict[str, Any] = {}
    if name is not None:
        data["name"] = name
    if email is not None:
        data["email"] = email
    if phone is not None:
        data["phone"] = phone
    if status is not None:
        data["status"] = status

    return update_customer({
        "customer_id": customer_id,
        "data": data,
    })

def mcp_create_ticket(customer_id: int, issue: str, priority: str = "medium") -> Dict[str, Any]:
    """Adapter for create_ticket(args)."""
    return create_ticket({
        "customer_id": customer_id,
        "issue": issue,
        "priority": priority,
    })

def mcp_get_customer_history(customer_id: int) -> Dict[str, Any]:
    """Adapter for get_customer_history(args)."""
    return get_customer_history({"customer_id": customer_id})

# ==================================
# tools/call handler
# ==================================

def handle_tools_call(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/call request.
    Executes the requested tool and returns the result.
    """
    params = message.get("params", {})
    tool_name = params.get("name")
    arguments = params.get("arguments", {})

    # Map tool names to adapter functions
    tool_functions = {
        "get_customer": mcp_get_customer,
        "list_customers": mcp_list_customers,
        "update_customer": mcp_update_customer,
        "create_ticket": mcp_create_ticket,
        "get_customer_history": mcp_get_customer_history,
    }

    if tool_name not in tool_functions:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Tool not found: {tool_name}"
            }
        }

    try:
        result = tool_functions[tool_name](**arguments)

        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "result": {
                "content": [
                    {
                        "type": "text",
                        "text": json.dumps(result, indent=2)
                    }
                ]
            }
        }
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32603,
                "message": f"Tool execution error: {str(e)}"
            }
        }

# ===================
# MCP message router
# ===================

def process_mcp_message(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Process an MCP message and route it to the appropriate handler.
    """
    method = message.get("method")

    if method == "initialize":
        return handle_initialize(message)
    elif method == "tools/list":
        return handle_tools_list(message)
    elif method == "tools/call":
        return handle_tools_call(message)
    else:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Method not found: {method}"
            }
        }

# =====================
# Flask Routes (SSE)
# =====================

@app.route('/mcp', methods=['POST'])
def mcp_endpoint():
    """
    Main MCP endpoint for MCP communication.
    Receives MCP messages and streams responses using Server-Sent Events.
    """
    message = request.get_json()

    def generate():
        try:
            print(f"[INFO] Received MCP message: {message.get('method')}")
            response = process_mcp_message(message)
            print("[INFO] Sending MCP response")
            yield create_sse_message(response)
        except Exception as e:
            error_response = {
                "jsonrpc": "2.0",
                "id": message.get("id"),
                "error": {
                    "code": -32700,
                    "message": f"Parse error: {str(e)}"
                }
            }
            yield create_sse_message(error_response)

    return Response(generate(), mimetype='text/event-stream')

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint to verify server is running."""
    return jsonify({
        "status": "healthy",
        "server": "customer-support-mcp-server",
        "version": "1.0.0"
    })

print("[SUCCESS] MCP Server implementation complete!")
print(f"   - {len(MCP_TOOLS)} tools exposed:")
for t in MCP_TOOLS:
    print("     •", t["name"])


[SUCCESS] MCP Server implementation complete!
   - 5 tools exposed:
     • get_customer
     • list_customers
     • update_customer
     • create_ticket
     • get_customer_history


## Start the MCP Server

In [9]:
HOST = "127.0.0.1"
PORT = 8000
SERVER_URL = f"http://{HOST}:{PORT}"

def run_server():
    """Run Flask server (blocking). Should be used inside a thread."""
    app.run(host=HOST, port=PORT, debug=False, use_reloader=False)

def start_server():
    """Start the MCP server in a background thread and wait for /health OK."""
    global server_thread, server_running

    if server_running:
        print("[INFO] Server is already running.")
        return

    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    # Wait for health check to be ready
    print("[INFO] Starting server...")
    for i in range(30):
        try:
            r = requests.get(f"{SERVER_URL}/health", timeout=1)
            if r.status_code == 200:
                server_running = True
                print(f"[INFO] Server is up at {SERVER_URL}")
                print(f"   MCP endpoint: {SERVER_URL}/mcp")
                return
        except Exception:
            time.sleep(0.5)

    print("[ERROR] Failed to detect running server via /health")

def check_server_status():
    """Quick /health probe."""
    try:
        r = requests.get(f"{SERVER_URL}/health", timeout=2)
        print("Health:", r.status_code, r.json())
    except Exception as e:
        print("[ERROR] Health check failed:", e)

In [10]:
start_server()
check_server_status()

[INFO] Starting server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:05] "GET /health HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:05] "GET /health HTTP/1.1" 200 -


[INFO] Server is up at http://127.0.0.1:8000
   MCP endpoint: http://127.0.0.1:8000/mcp
Health: 200 {'server': 'customer-support-mcp-server', 'status': 'healthy', 'version': '1.0.0'}


In [11]:
def send_mcp_message(method: str, params: Dict[str, Any] = None, message_id: int = 1):
    """
    Send a single MCP message to /mcp and print/return the response.
    Uses SSE (Server-Sent Events)
    """
    if params is None:
        params = {}

    message = {
        "jsonrpc": "2.0",
        "id": message_id,
        "method": method,
        "params": params,
    }

    print("\n==============================")
    print(f"[INFO] MCP request → {method}")
    print(json.dumps(message, indent=2))

    resp = requests.post(
        f"{SERVER_URL}/mcp",
        json=message,
        stream=True,
        timeout=30,
    )

    if resp.status_code != 200:
        print("[ERROR] HTTP error:", resp.status_code, resp.text)
        return None

    full_payload = None

    print("[INFO] MCP response:")
    for line in resp.iter_lines(decode_unicode=True):
        if not line:
            continue
        if line.startswith("data: "):
            data_str = line[len("data: "):]
            try:
                payload = json.loads(data_str)
                full_payload = payload
                print(json.dumps(payload, indent=2))
            except Exception as e:
                print("[ERROR] Failed to parse JSON from SSE line:", e, "\nRaw:", data_str)

    return full_payload

In [12]:
class MCPToolClient:
    """
    Host-side client that talks to your MCP server via /mcp.
    Provides a simple call_tool(name, arguments) interface.
    """

    def __init__(self, server_url: str):
        self.server_url = server_url
        self._next_id = 1

    def _send_mcp_message(self, method: str, params: Dict[str, Any]) -> Dict[str, Any]:
        message_id = self._next_id
        self._next_id += 1

        message = {
            "jsonrpc": "2.0",
            "id": message_id,
            "method": method,
            "params": params,
        }

        resp = requests.post(
            f"{self.server_url}/mcp",
            json=message,
            stream=True,
            timeout=30,
        )

        if resp.status_code != 200:
            raise RuntimeError(f"MCP HTTP error: {resp.status_code} {resp.text}")

        full_payload = None
        for line in resp.iter_lines(decode_unicode=True):
            if not line:
                continue
            if line.startswith("data: "):
                data_str = line[len("data: "):]
                full_payload = json.loads(data_str)
                break

        if full_payload is None:
            raise RuntimeError("No SSE data received from MCP server")

        # If it's an error, raise
        if "error" in full_payload:
            raise RuntimeError(f"MCP error: {full_payload['error']}")

        return full_payload

    def call_tool(self, name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Call an MCP tool and return the parsed JSON result produced
        by your tool function (get_customer / list_customers / etc.).
        """
        payload = self._send_mcp_message(
            "tools/call",
            {
                "name": name,
                "arguments": arguments,
            },
        )

        # packed tool result as text JSON in handle_tools_call
        content_items = payload.get("result", {}).get("content", [])
        if not content_items:
            return {}

        text_json = content_items[0].get("text", "{}")
        return json.loads(text_json)

## Test Section: MCP Protocol in Action

### TEST 1 – initialize

In [13]:
init_result = send_mcp_message(
    "initialize",
    {
        "protocolVersion": "2024-11-05",
        "capabilities": {}
    },
    message_id=1
)

INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:05] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → initialize
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "initialize",
  "params": {
    "protocolVersion": "2024-11-05",
    "capabilities": {}
  }
}
[INFO] Received MCP message: initialize
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "protocolVersion": "2024-11-05",
    "capabilities": {
      "tools": {}
    },
    "serverInfo": {
      "name": "customer-support-mcp-server",
      "version": "1.0.0"
    }
  }
}


### TEST 2 – tools/list

In [14]:
tools_result = send_mcp_message(
    "tools/list",
    {},
    message_id=2
)

INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:05] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → tools/list
{
  "jsonrpc": "2.0",
  "id": 2,
  "method": "tools/list",
  "params": {}
}
[INFO] Received MCP message: tools/list
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "tools": [
      {
        "name": "get_customer",
        "description": "Retrieve a specific customer by their ID. Returns customer details including name, email, phone, and status.",
        "inputSchema": {
          "type": "object",
          "properties": {
            "customer_id": {
              "type": "integer",
              "description": "The unique ID of the customer to retrieve"
            }
          },
          "required": [
            "customer_id"
          ]
        }
      },
      {
        "name": "list_customers",
        "description": "List customers in the database. Can optionally filter by status and limit the number of results.",
        "inputSchema": {
          "type": "object",
          "properties": {

### TEST 3 – tools/call

In [15]:
get_cust_raw = send_mcp_message(
    "tools/call",
    {
        "name": "get_customer",
        "arguments": {
            "customer_id": 1
        }
    },
    message_id=3
)

if get_cust_raw and "result" in get_cust_raw:
    text_payload = get_cust_raw["result"]["content"][0]["text"]
    get_cust_data = json.loads(text_payload)
    print("\n Parsed get_customer result:")
    print(json.dumps(get_cust_data, indent=2))


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:05] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → tools/call
{
  "jsonrpc": "2.0",
  "id": 3,
  "method": "tools/call",
  "params": {
    "name": "get_customer",
    "arguments": {
      "customer_id": 1
    }
  }
}
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 3,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"data\": {\n    \"id\": 1,\n    \"name\": \"John Doe\",\n    \"email\": \"john.new@example.com\",\n    \"phone\": \"+1-555-7777\",\n    \"status\": \"active\",\n    \"created_at\": \"2025-11-30 20:26:34\",\n    \"updated_at\": \"2025-12-01 01:57:04\"\n  }\n}"
      }
    ]
  }
}

 Parsed get_customer result:
{
  "success": true,
  "data": {
    "id": 1,
    "name": "John Doe",
    "email": "john.new@example.com",
    "phone": "+1-555-7777",
    "status": "active",
    "created_at": "2025-11-30 20:26:34",
    "updated_at": "2025-12-01 01:57:04"
  }
}


In [16]:
list_raw = send_mcp_message(
    "tools/call",
    {
        "name": "list_customers",
        "arguments": {
            "status": "active",
            "limit": 5
        }
    },
    message_id=4
)

if list_raw and "result" in list_raw:
    text_payload = list_raw["result"]["content"][0]["text"]
    list_data = json.loads(text_payload)
    print("\n Parsed list_customers result:")
    print(json.dumps(list_data, indent=2))


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:05] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → tools/call
{
  "jsonrpc": "2.0",
  "id": 4,
  "method": "tools/call",
  "params": {
    "name": "list_customers",
    "arguments": {
      "status": "active",
      "limit": 5
    }
  }
}
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 4,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"count\": 5,\n  \"customers\": [\n    {\n      \"id\": 1,\n      \"name\": \"John Doe\",\n      \"email\": \"john.new@example.com\",\n      \"phone\": \"+1-555-7777\",\n      \"status\": \"active\",\n      \"created_at\": \"2025-11-30 20:26:34\",\n      \"updated_at\": \"2025-12-01 01:57:04\"\n    },\n    {\n      \"id\": 2,\n      \"name\": \"Jane Smith\",\n      \"email\": \"jane.smith@example.com\",\n      \"phone\": \"+1-555-0102\",\n      \"status\": \"active\",\n      \"created_at\": \"2025-11-30 20:26:34\",\n      \"updated_at\": \"2025-11-30 20:2

In [17]:
# create_ticket
ticket_raw = send_mcp_message(
    "tools/call",
    {
        "name": "create_ticket",
        "arguments": {
            "customer_id": 1,
            "issue": "My order arrived damaged",
            "priority": "high"
        }
    },
    message_id=5
)

# get_customer_history
history_raw = send_mcp_message(
    "tools/call",
    {
        "name": "get_customer_history",
        "arguments": {
            "customer_id": 1
        }
    },
    message_id=6
)

if history_raw and "result" in history_raw:
    text_payload = history_raw["result"]["content"][0]["text"]
    history_data = json.loads(text_payload)
    print("\n Parsed get_customer_history result:")
    print(json.dumps(history_data, indent=2))



[INFO] MCP request → tools/call
{
  "jsonrpc": "2.0",
  "id": 5,
  "method": "tools/call",
  "params": {
    "name": "create_ticket",
    "arguments": {
      "customer_id": 1,
      "issue": "My order arrived damaged",
      "priority": "high"
    }
  }
}
[INFO] Received MCP message: tools/call


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:05] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:05] "POST /mcp HTTP/1.1" 200 -


[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 5,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"ticket\": {\n    \"id\": 87,\n    \"customer_id\": 1,\n    \"issue\": \"My order arrived damaged\",\n    \"status\": \"open\",\n    \"priority\": \"high\",\n    \"created_at\": \"2025-12-01 01:57:05\"\n  }\n}"
      }
    ]
  }
}

[INFO] MCP request → tools/call
{
  "jsonrpc": "2.0",
  "id": 6,
  "method": "tools/call",
  "params": {
    "name": "get_customer_history",
    "arguments": {
      "customer_id": 1
    }
  }
}
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 6,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"count\": 16,\n  \"tickets\": [\n    {\n      \"id\": 87,\n      \"customer_id\": 1,\n      \"issue\": \"My order arrived damaged\",\n      \"status\": \

## A2A

In [18]:
import os
import json
import requests

OPENROUTER_API_KEY = "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"

OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
DEFAULT_OPENROUTER_MODEL = "x-ai/grok-4.1-fast:free"

def generate_local_llm(prompt: str, max_new_tokens: int = 512) -> str:
    """
    Generate text using a remote LLM via OpenRouter instead of a local model.

    """
    if not OPENROUTER_API_KEY:
        raise RuntimeError("OPENROUTER_API_KEY env var is not set")

    payload = {
        "model": DEFAULT_OPENROUTER_MODEL,
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ],
        "max_tokens": max_new_tokens,
        "temperature": 0.0,
        "top_p": 1.0,
        "reasoning": {"enabled": True},
    }

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
    }

    resp = requests.post(
        OPENROUTER_URL,
        headers=headers,
        data=json.dumps(payload),
        timeout=60,
    )
    resp.raise_for_status()
    data = resp.json()

    # OpenRouter normalizes to OpenAI chat schema: choices[0].message.content
    text = data["choices"][0]["message"]["content"]
    return text



ROUTER_SYSTEM_PROMPT = """
You are the Router agent in a multi-agent customer-service system.

You DO NOT call tools yourself. Your ONLY job is to plan which internal agents
should be invoked, and with what message types and payloads.

Available agents (targets) and what they do:

1) "customer_data"
   - kind: "get_customer"
     payload: {"customer_id": int}
   - kind: "list_customers"
     payload: {
       "status": "active" | "disabled" | null,
       "limit": int,
       "for_open_tickets"?: bool
     }
   - kind: "update_customer"
     payload: {
       "customer_id": int,
       "data": {"email"?: str, "name"?: str, "phone"?: str, "status"?: str},
       "multi_intent"?: bool,
       "multi_key"?: str
     }

2) "support"
   - kind: "support_request"
     payload: {
       "customer_id": int,
       "issue": str,
       "priority": "low" | "medium" | "high",
       "urgent"?: bool
     }
   - kind: "history_only"
     payload: {
       "customer_id": int,
       "multi_intent"?: bool,
       "multi_key"?: str
     }

The user query may have one or multiple intents.

You MUST return ONLY a single JSON object in this exact format:

{
  "tasks": [
    {
      "target": "customer_data" | "support",
      "kind": "get_customer" | "list_customers" | "update_customer" |
              "support_request" | "history_only",
      "payload": { ... },
      "description": "short human-readable description"
    }
  ]
}

No extra keys. No commentary. No markdown. Only JSON.

Patterns:

- Simple Query: "Get customer information for ID 5"
  -> one task:
     {
       "target": "customer_data",
       "kind": "get_customer",
       "payload": {"customer_id": 5},
       "description": "Fetch customer by ID"
     }

- Coordinated Query: "I'm customer 12345 and need help upgrading my account"
  -> two tasks (in order):
     1) customer_data/get_customer with customer_id=12345
     2) support/support_request with
        customer_id=12345, issue="help upgrading my account", priority="medium"

- Complex Query: "Show me all active customers who have open tickets"
  -> ONE task to list active customers, marking it for open-ticket processing:
     {
       "target": "customer_data",
       "kind": "list_customers",
       "payload": {
         "status": "active",
         "limit": 100,
         "for_open_tickets": true
       },
       "description": "List active customers for open-ticket analysis"
     }

- Escalation: "I've been charged twice, please refund immediately!"
  -> ONE support_request with high priority and urgent=true:
     {
       "target": "support",
       "kind": "support_request",
       "payload": {
         "customer_id": <id>,
         "issue": "I've been charged twice, please refund immediately!",
         "priority": "high",
         "urgent": true
       },
       "description": "Urgent billing escalation"
     }

- Multi-Intent: "Update my email to X and show my ticket history"
  -> TWO tasks in parallel for the same customer:
     {
       "target": "customer_data",
       "kind": "update_customer",
       "payload": {
         "customer_id": <id>,
         "data": {"email": X},
         "multi_intent": true,
         "multi_key": "customer-<id>"
       },
       "description": "Update customer email"
     }
     {
       "target": "support",
       "kind": "history_only",
       "payload": {
         "customer_id": <id>,
         "multi_intent": true,
         "multi_key": "customer-<id>"
       },
       "description": "Fetch ticket history for multi-intent flow"
     }

If customer_id is not mentioned in the text, use the one provided in the
input payload, or default to 1.

Remember: OUTPUT MUST BE VALID JSON WITH TOP-LEVEL "tasks" ARRAY ONLY.
"""

def llm_plan_for_router(user_text: str, payload: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Use the local Mistral-7B-LoRA model to produce a routing plan for the Router.

    Returns: list of tasks:
      [
        {"target": "...", "kind": "...", "payload": {...}, "description": "..."},
        ...
      ]
    """
    # Build prompt with system-style instructions and user context
    prompt = ROUTER_SYSTEM_PROMPT + "\n\n" + \
        "User message and payload:\n" + json.dumps({
            "query": user_text,
            "payload": payload,
        }, indent=2) + "\n\nReturn ONLY the JSON now:"

    raw_output = generate_local_llm(prompt, max_new_tokens=512)

    # Try to extract JSON from the model output
    try:
        # find first '{'
        start = raw_output.index("{")
        json_str = raw_output[start:]
        parsed = json.loads(json_str)
        tasks = parsed.get("tasks", [])
        if not isinstance(tasks, list):
            raise ValueError("Field 'tasks' is not a list")
        return tasks
    except Exception as e:
        print("[Router LLM] JSON parse error:", e)
        print("=== Raw LLM output ===")
        print(raw_output)
        print("=== end output ===")
        return []


# ============================================================
#  A2A Message and BaseAgent
# ============================================================

@dataclass
class AgentMessage:
    """
    A simple A2A message envelope for agents to talk to each other.
    """
    sender: str           # who sent this message (e.g. "router", "support")
    target: str           # who should handle this message (e.g. "customer_data")
    kind: str             # message type, e.g. "user_request", "get_customer"
    content: str          # human-readable description
    payload: Dict[str, Any] = field(default_factory=dict)


class BaseAgent:
    def __init__(self, name: str):
        self.name = name

    def handle(self, message: AgentMessage, tools: "MCPToolClient") -> List[AgentMessage]:
        """
        Process an incoming message and return zero or more outgoing messages.
        """
        raise NotImplementedError


# ============================================================
#  LLM-based RouterAgent
# ============================================================

class RouterAgent(BaseAgent):
    """
    Router agent that uses LLM model to decide which downstream agents to call and with what payloads.
    """

    def __init__(self):
        super().__init__(name="router")

    def handle(self, message: AgentMessage, tools: "MCPToolClient") -> List[AgentMessage]:
        user_text = message.content
        payload = message.payload or {}
        out: List[AgentMessage] = []

        print("\n[Router] Calling local LLM planner...")
        tasks = llm_plan_for_router(user_text, payload)

        if not tasks:
            # Very defensive fallback: just fetch customer info.
            customer_id = payload.get("customer_id", 1)
            out.append(AgentMessage(
                sender=self.name,
                target="customer_data",
                kind="get_customer",
                content="[fallback] Fetch customer info",
                payload={"customer_id": customer_id},
            ))
            return out

        for t in tasks:
            target = t.get("target")
            kind = t.get("kind")
            desc = t.get("description", f"{kind} via LLM plan")
            pl = t.get("payload", {})

            if target not in {"customer_data", "support"} or not kind:
                print("[Router LLM] Skipping invalid task:", t)
                continue

            out.append(AgentMessage(
                sender=self.name,
                target=target,
                kind=kind,
                content=desc,
                payload=pl,
            ))

        return out


# ============================================================
#  CustomerDataAgent
# ============================================================

class CustomerDataAgent(BaseAgent):
    """
    Agent specialized in talking to the MCP tools that read/write customer data.
    """

    def __init__(self):
        super().__init__(name="customer_data")

    def handle(self, message: AgentMessage, tools: "MCPToolClient") -> List[AgentMessage]:
        kind = message.kind
        out: List[AgentMessage] = []

        # ---- Single customer lookup ----
        if kind == "get_customer":
            cid = message.payload["customer_id"]
            result = tools.call_tool("get_customer", {"customer_id": cid})

            out.append(AgentMessage(
                sender=self.name,
                target="support",
                kind="customer_info",
                content=f"Customer info for ID {cid}",
                payload=result,
            ))

        # ---- Customer listing (possibly for complex scenario) ----
        elif kind == "list_customers":
            status = message.payload.get("status")
            limit = message.payload.get("limit", 5)
            for_open_tickets = message.payload.get("for_open_tickets", False)

            result = tools.call_tool("list_customers", {
                "status": status,
                "limit": limit,
            })

            if for_open_tickets:
                # Special path for the complex query:
                # hand off directly to SupportAgent for further processing.
                out.append(AgentMessage(
                    sender=self.name,
                    target="support",
                    kind="customers_for_open_tickets",
                    content="Active customers for open-ticket query.",
                    payload=result,
                ))
            else:
                # Normal path: return the list to whoever asked (router/user/etc.)
                out.append(AgentMessage(
                    sender=self.name,
                    target=message.sender,
                    kind="customers_list",
                    content=f"Listed {result.get('count', 0)} customers",
                    payload=result,
                ))

        # ---- Update customer (for multi-intent scenario, or general use) ----
        elif kind == "update_customer":
            cid = message.payload["customer_id"]
            data = message.payload.get("data", {})
            multi_intent = message.payload.get("multi_intent", False)
            multi_key = message.payload.get("multi_key")

            # Flatten for mcp_update_customer(customer_id, name, email, phone, status)
            tool_args = {"customer_id": cid}
            tool_args.update(data)

            result = tools.call_tool("update_customer", tool_args)

            if multi_intent:
                out.append(AgentMessage(
                    sender=self.name,
                    target="support",
                    kind="update_done",
                    content=f"Update completed for customer {cid}",
                    payload={
                        "customer_id": cid,
                        "update_result": result,
                        "multi_intent": True,
                        "multi_key": multi_key,
                    },
                ))
            else:
                out.append(AgentMessage(
                    sender=self.name,
                    target="user",
                    kind="final_answer",
                    content=f"Updated customer {cid} successfully.",
                    payload=result,
                ))

        # Unknown kind => no-op
        return out

# ============================================================
#  SupportAgent
# ============================================================

class SupportAgent(BaseAgent):
    """
    Agent that owns support logic: deciding when to open tickets,
    fetching ticket history, and building reports.
    """

    def __init__(self):
        super().__init__(name="support")
        # For multi-intent coordination: store partial results keyed by multi_key
        self.multi_intent_buffer: Dict[str, Dict[str, Any]] = {}

    def _update_multi_buffer(self, multi_key: str, part: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        """
        Helper for multi-intent: merge partial results and decide
        whether to return a combined final_answer.
        """
        if multi_key not in self.multi_intent_buffer:
            self.multi_intent_buffer[multi_key] = {}

        self.multi_intent_buffer[multi_key].update(part)
        bundle = self.multi_intent_buffer[multi_key]

        # For multi-intent case, need both "update_result" and "history_result"
        if "update_result" in bundle and "history_result" in bundle:
            completed = self.multi_intent_buffer.pop(multi_key)
            return completed

        return None

    def handle(self, message: AgentMessage, tools: "MCPToolClient") -> List[AgentMessage]:
        kind = message.kind
        out: List[AgentMessage] = []

        # ---- Generic / escalated support request: open ticket + history ----
        if kind == "support_request":
            customer_id = message.payload.get("customer_id", 1)
            issue = message.payload.get("issue", "User reported an issue.")
            priority = message.payload.get("priority", "medium")
            urgent = message.payload.get("urgent", False)

            ticket_result = tools.call_tool("create_ticket", {
                "customer_id": customer_id,
                "issue": issue,
                "priority": priority,
            })

            history_result = tools.call_tool("get_customer_history", {
                "customer_id": customer_id,
            })

            prefix = "Escalated high-priority issue" if urgent else "Created ticket"
            out.append(AgentMessage(
                sender=self.name,
                target="user",
                kind="final_answer",
                content=f"{prefix} for customer {customer_id} (priority={priority}) and fetched history.",
                payload={
                    "ticket": ticket_result,
                    "history": history_result,
                },
            ))

        # ---- Single customer info (from CustomerDataAgent) ----
        elif kind == "customer_info":
            out.append(AgentMessage(
                sender=self.name,
                target="user",
                kind="final_answer",
                content="Here is the customer info:",
                payload=message.payload,
            ))

        # ---- Complex scenario: active customers with open tickets ----
        elif kind == "customers_for_open_tickets":
            customers = message.payload.get("customers", [])
            customers_with_open = []

            for cust in customers:
                cid = cust.get("id")
                if cid is None:
                    continue

                history = tools.call_tool("get_customer_history", {
                    "customer_id": cid,
                })
                tickets = history.get("tickets", [])
                open_tickets = [t for t in tickets if t.get("status") == "open"]

                if open_tickets:
                    customers_with_open.append({
                        "customer": cust,
                        "open_tickets": open_tickets,
                    })

            out.append(AgentMessage(
                sender=self.name,
                target="user",
                kind="final_answer",
                content=f"Found {len(customers_with_open)} active customers with open tickets.",
                payload={
                    "customers_with_open_tickets": customers_with_open
                },
            ))

        # ---- Multi-intent: history-only part (no new ticket) ----
        elif kind == "history_only":
            customer_id = message.payload.get("customer_id", 1)
            multi_intent = message.payload.get("multi_intent", False)
            multi_key = message.payload.get("multi_key")

            history_result = tools.call_tool("get_customer_history", {
                "customer_id": customer_id,
            })

            if multi_intent and multi_key:
                completed = self._update_multi_buffer(
                    multi_key,
                    {"history_result": history_result, "customer_id": customer_id},
                )
                if completed is not None:
                    out.append(AgentMessage(
                        sender=self.name,
                        target="user",
                        kind="final_answer",
                        content=f"Updated email and fetched ticket history for customer {customer_id}.",
                        payload={
                            "update": completed["update_result"],
                            "history": completed["history_result"],
                        },
                    ))
            else:
                out.append(AgentMessage(
                    sender=self.name,
                    target="user",
                    kind="final_answer",
                    content=f"Here is the ticket history for customer {customer_id}.",
                    payload={"history": history_result},
                ))

        # ---- Multi-intent: update part completed ----
        elif kind == "update_done":
            customer_id = message.payload.get("customer_id", 1)
            multi_intent = message.payload.get("multi_intent", False)
            multi_key = message.payload.get("multi_key")
            update_result = message.payload.get("update_result")

            if multi_intent and multi_key:
                completed = self._update_multi_buffer(
                    multi_key,
                    {"update_result": update_result, "customer_id": customer_id},
                )
                if completed is not None:
                    out.append(AgentMessage(
                        sender=self.name,
                        target="user",
                        kind="final_answer",
                        content=f"Updated email and fetched ticket history for customer {customer_id}.",
                        payload={
                            "update": completed["update_result"],
                            "history": completed["history_result"],
                        },
                    ))
            else:
                out.append(AgentMessage(
                    sender=self.name,
                    target="user",
                    kind="final_answer",
                    content=f"Customer {customer_id} updated successfully.",
                    payload={"update": update_result},
                ))

        return out


# ============================================================
#  Multi Agent Orchestrator
# ============================================================

class MultiAgentOrchestrator:

    def __init__(self, tools: "MCPToolClient"):
        self.tools = tools
        self.router = RouterAgent()
        self.customer_data = CustomerDataAgent()
        self.support = SupportAgent()

    def dispatch(self, msg: AgentMessage) -> List[AgentMessage]:
        print(f"[DISPATCH] {msg.sender} -> {msg.target} ({msg.kind})")
        if msg.target == "router":
            return self.router.handle(msg, self.tools)
        elif msg.target == "customer_data":
            return self.customer_data.handle(msg, self.tools)
        elif msg.target == "support":
            return self.support.handle(msg, self.tools)
        else:
            return []

    def run(self, user_text: str, payload: Optional[Dict[str, Any]] = None) -> List[AgentMessage]:
        if payload is None:
            payload = {}

        queue: List[AgentMessage] = [
            AgentMessage(
                sender="user",
                target="router",
                kind="user_request",
                content=user_text,
                payload=payload,
            )
        ]
        final_answers: List[AgentMessage] = []

        while queue:
            current = queue.pop(0)
            print(f"[QUEUE POP] target={current.target}, kind={current.kind}")
            if current.target == "user":
                print(f"[FINAL] {current.sender} -> user ({current.kind})")
                final_answers.append(current)
                continue

            new_msgs = self.dispatch(current)
            queue.extend(new_msgs)

        return final_answers

print("LLM-based Multi-Agent A2A system (Router / CustomerData / Support) defined.")

LLM-based Multi-Agent A2A system (Router / CustomerData / Support) defined.


In [19]:
tools_client = MCPToolClient(SERVER_URL)
orchestrator = MultiAgentOrchestrator(tools_client)

## TEST Scenarios

### TEST 1 - Simple Query

In [20]:
print("=== Scenario: Simple Query – Get customer information for ID 5 ===")
answers = orchestrator.run(
    "Get customer information for ID 5",
    payload={"customer_id": 5}
)

for msg in answers:
    print("\n--- Final AgentMessage ---")
    print("sender:", msg.sender)
    print("kind:", msg.kind)
    print("content:", msg.content)
    print("payload:")
    print(json.dumps(msg.payload, indent=2))


=== Scenario: Simple Query – Get customer information for ID 5 ===
[QUEUE POP] target=router, kind=user_request
[DISPATCH] user -> router (user_request)

[Router] Calling local LLM planner...


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:11] "POST /mcp HTTP/1.1" 200 -


[QUEUE POP] target=customer_data, kind=get_customer
[DISPATCH] router -> customer_data (get_customer)
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[QUEUE POP] target=support, kind=customer_info
[DISPATCH] customer_data -> support (customer_info)
[QUEUE POP] target=user, kind=final_answer
[FINAL] support -> user (final_answer)

--- Final AgentMessage ---
sender: support
kind: final_answer
content: Here is the customer info:
payload:
{
  "success": true,
  "data": {
    "id": 5,
    "name": "Charlie Brown",
    "email": "new@email.com",
    "phone": "+1-555-0105",
    "status": "active",
    "created_at": "2025-11-30 20:26:34",
    "updated_at": "2025-11-30 22:59:02"
  }
}


### TEST 2 - Coordinated Query

In [21]:
print("=== Scenario: Coordinated Query – Upgrade Account ===")
query_text = "I'm customer 5 and need help upgrading my account"

answers = orchestrator.run(
    query_text,
    payload={
        "customer_id": 5,
        "issue": "Customer wants to upgrade account",
        "priority": "medium",
    }
)

for msg in answers:
    print("\n--- Final AgentMessage ---")
    print("sender:", msg.sender)
    print("kind:", msg.kind)
    print("content:", msg.content)
    print("payload:")
    print(json.dumps(msg.payload, indent=2))


=== Scenario: Coordinated Query – Upgrade Account ===
[QUEUE POP] target=router, kind=user_request
[DISPATCH] user -> router (user_request)

[Router] Calling local LLM planner...


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:24] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:24] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:24] "POST /mcp HTTP/1.1" 200 -


[QUEUE POP] target=customer_data, kind=get_customer
[DISPATCH] router -> customer_data (get_customer)
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[QUEUE POP] target=support, kind=support_request
[DISPATCH] router -> support (support_request)
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[QUEUE POP] target=support, kind=customer_info
[DISPATCH] customer_data -> support (customer_info)
[QUEUE POP] target=user, kind=final_answer
[FINAL] support -> user (final_answer)
[QUEUE POP] target=user, kind=final_answer
[FINAL] support -> user (final_answer)

--- Final AgentMessage ---
sender: support
kind: final_answer
content: Created ticket for customer 5 (priority=medium) and fetched history.
payload:
{
  "ticket": {
    "success": true,
    "ticket": {
      "id": 88,
      "customer_id": 5,
      "issue": "need help upgrading my account",
      "status": "open",
      "priority": "

### TEST 3 - Complex Query

In [22]:
print("=== Scenario: Complex Query – Active customers with open tickets ===")
answers = orchestrator.run(
    "Show me all active customers who have open tickets",
    payload={}
)

for msg in answers:
    print("\n--- Final AgentMessage ---")
    print("sender:", msg.sender)
    print("kind:", msg.kind)
    print("content:", msg.content)
    print("payload:")
    print(json.dumps(msg.payload, indent=2))


=== Scenario: Complex Query – Active customers with open tickets ===
[QUEUE POP] target=router, kind=user_request
[DISPATCH] user -> router (user_request)

[Router] Calling local LLM planner...


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:27] "POST /mcp HT

[QUEUE POP] target=customer_data, kind=list_customers
[DISPATCH] router -> customer_data (list_customers)
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[QUEUE POP] target=support, kind=customers_for_open_tickets
[DISPATCH] customer_data -> support (customers_for_open_tickets)
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP m

### TEST 4 - Escalation

In [23]:
print("=== Scenario: Escalation – Urgent refund ===")
answers = orchestrator.run(
    "I've been charged twice, please refund immediately!",
    payload={"customer_id": 1}
)

for msg in answers:
    print("\n--- Final AgentMessage ---")
    print("sender:", msg.sender)
    print("kind:", msg.kind)
    print("content:", msg.content)
    print("payload:")
    print(json.dumps(msg.payload, indent=2))


=== Scenario: Escalation – Urgent refund ===
[QUEUE POP] target=router, kind=user_request
[DISPATCH] user -> router (user_request)

[Router] Calling local LLM planner...


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:32] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:32] "POST /mcp HTTP/1.1" 200 -


[QUEUE POP] target=support, kind=support_request
[DISPATCH] router -> support (support_request)
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[QUEUE POP] target=user, kind=final_answer
[FINAL] support -> user (final_answer)

--- Final AgentMessage ---
sender: support
kind: final_answer
content: Escalated high-priority issue for customer 1 (priority=high) and fetched history.
payload:
{
  "ticket": {
    "success": true,
    "ticket": {
      "id": 89,
      "customer_id": 1,
      "issue": "I've been charged twice, please refund immediately!",
      "status": "open",
      "priority": "high",
      "created_at": "2025-12-01 01:57:32"
    }
  },
  "history": {
    "success": true,
    "count": 17,
    "tickets": [
      {
        "id": 89,
        "customer_id": 1,
        "issue": "I've been charged twice, please refund immediately!",
        "status": "open",
        "priority": "high",
        "

### TEST 5 - Multi-Intent

In [24]:
print("=== Scenario: Multi-Intent – Update email + show ticket history ===")
answers = orchestrator.run(
    "Update my email to new@email.com and show my ticket history",
    payload={"customer_id": 5}
)

for msg in answers:
    print("\n--- Final AgentMessage ---")
    print("sender:", msg.sender)
    print("kind:", msg.kind)
    print("content:", msg.content)
    print("payload:")
    print(json.dumps(msg.payload, indent=2))


=== Scenario: Multi-Intent – Update email + show ticket history ===
[QUEUE POP] target=router, kind=user_request
[DISPATCH] user -> router (user_request)

[Router] Calling local LLM planner...


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:38] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 01:57:38] "POST /mcp HTTP/1.1" 200 -


[QUEUE POP] target=customer_data, kind=update_customer
[DISPATCH] router -> customer_data (update_customer)
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[QUEUE POP] target=support, kind=history_only
[DISPATCH] router -> support (history_only)
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[QUEUE POP] target=support, kind=update_done
[DISPATCH] customer_data -> support (update_done)
[QUEUE POP] target=user, kind=final_answer
[FINAL] support -> user (final_answer)

--- Final AgentMessage ---
sender: support
kind: final_answer
content: Updated email and fetched ticket history for customer 5.
payload:
{
  "update": {
    "success": true,
    "data": {
      "id": 5,
      "name": "Charlie Brown",
      "email": "new@email.com",
      "phone": "+1-555-0105",
      "status": "active",
      "created_at": "2025-11-30 20:26:34",
      "updated_at": "2025-12-01 01:57:38"
    }
  },
  "history": {
    "success": true,
    "count": 9,
    "tickets": [
 

## Conclusion

Through building this multi-agent customer service system, I learned how Agent-to-Agent (A2A) coordination, Model Context Protocol (MCP), and an LLM-driven Router can work together to form a fully autonomous workflow. Designing the Router to generate structured JSON tasks was especially valuable, because it enabled the system to generalize across simple queries, escalations, multi-intent requests, and complex analytical tasks. I also gained practical experience integrating an external model (Grok-4.1 via OpenRouter) into a real agentic loop and using MCP tools as a secure, standardized interface to external data (customer records, ticketing DB, etc.).

The biggest challenges came from making the system robust to imperfect LLM output and ensuring multi-intent flows could run in parallel without losing consistency. I had to design a merge buffer, enforce strict JSON schemas, sanitize LLM outputs, and carefully orchestrate how different agents exchanged intermediate results. Overall, the project taught me how to combine LLM reasoning, structured APIs, and agent autonomy into a cohesive and extensible system capable of realistic customer-support tasks.